> 请在环境变量中设置`DB_URI`指向数据库

In [1]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from alphamind.api import *
from alphamind.strategy.strategy import RunningSetting
from alphamind.strategy.strategy import Strategy
from PyFin.api import *
from PyFin.Math.Accumulators.StatefulAccumulators import MovingAverage
from PyFin.Math.Accumulators.StatefulAccumulators import MovingSharp
from PyFin.Math.Accumulators.StatefulAccumulators import MovingMaxDrawdown

plt.style.use('ggplot')

In [2]:
"""
Back test parameter settings
"""

benchmark_code = 905
universe = Universe('zz800') + Universe('cyb')

start_date = '2017-01-01'
end_date = '2018-04-27'
freq = '10b'
neutralized_risk = None

alpha_factors = {
    'f01': CSQuantiles(LAST('EPS'), groups='sw1'),
    'f02': CSQuantiles(LAST('ROE'), groups='sw1'),
}

weights = dict(
               f01=1.,
               f02=1.
               )

alpha_model = ConstLinearModel(features=alpha_factors, weights=weights)

data_meta = DataMeta(freq=freq,
                     universe=universe,
                     batch=1,
                     neutralized_risk=None,
                     pre_process=None,
                     post_process=None,
                     data_source=os.environ['DB_URI'])

strategy = Strategy(alpha_model,
                    data_meta,
                    universe=universe,
                    start_date=start_date,
                    end_date=end_date,
                    freq=freq,
                    benchmark=benchmark_code)

strategy.prepare_backtest_data()

def create_scenario(weights_bandwidth=0.02, target_vol=0.01, method='risk_neutral'):
    industry_names = industry_list('sw', 1)
    constraint_risk = ['SIZE', 'SIZENL', 'BETA']
    total_risk_names = constraint_risk + ['benchmark', 'total']
    all_styles = risk_styles + industry_styles + macro_styles

    b_type = []
    l_val = []
    u_val = []

    previous_pos = pd.DataFrame()
    rets = []
    turn_overs = []
    leverags = []

    for name in total_risk_names:
        if name == 'benchmark':
            b_type.append(BoundaryType.RELATIVE)
            l_val.append(0.8)
            u_val.append(1.0)
        else:
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(0.0)
            u_val.append(0.0)

    bounds = create_box_bounds(total_risk_names, b_type, l_val, u_val)
    running_setting = RunningSetting(weights_bandwidth=weights_bandwidth,
                                     rebalance_method=method,
                                     bounds=bounds,
                                     target_vol=target_vol,
                                     turn_over_target=0.4)

    ret_df, positions = strategy.run(running_setting)
    return ret_df

2019-02-10 00:34:06,090 - ALPHA_MIND - INFO - alpha factor data loading finished ...
2019-02-10 00:34:06,861 - ALPHA_MIND - INFO - industry data loading finished ...
2019-02-10 00:34:07,265 - ALPHA_MIND - INFO - benchmark data loading finished ...
2019-02-10 00:34:10,637 - ALPHA_MIND - INFO - risk_model data loading finished ...
2019-02-10 00:34:13,350 - ALPHA_MIND - INFO - returns data loading finished ...


In [3]:
def create_report(ret_df, windows):
    sharp_calc = MovingSharp(windows, x='ret', y='riskFree')
    drawdown_calc = MovingMaxDrawdown(windows, x='ret')
    max_drawdown_calc = MovingMaxDrawdown(len(ret_df), x='ret')
    
    ret_df['ret_after_tc'] = ret_df.excess_return - 0.002 * ret_df.turn_over
    res_df = pd.DataFrame(columns=['daily_return', 'cum_ret', 'sharp', 'drawdown', 'max_drawn', 'leverage'])
    total_returns = 0.

    for i, ret in enumerate(ret_df['ret_after_tc']):
        date = ret_df.index[i]
        total_returns += ret
        sharp_calc.push({'ret': ret, 'riskFree': 0.})
        drawdown_calc.push({'ret': ret})
        max_drawdown_calc.push({'ret': ret})

        res_df.loc[date, 'daily_return'] = ret
        res_df.loc[date, 'cum_ret'] = total_returns
        res_df.loc[date, 'drawdown'] = drawdown_calc.result()
        res_df.loc[date, 'max_drawn'] = max_drawdown_calc.result()
        res_df.loc[date, 'leverage'] = ret_df.loc[date, 'leverage']

        if i < 5:
            res_df.loc[date, 'sharp'] = 0.
        else:
            res_df.loc[date, 'sharp'] = sharp_calc.result() * np.sqrt(windows)
    return res_df

In [4]:
%%time
weight_gaps = [0.005, 0.010, 0.015, 0.020]

with pd.ExcelWriter(f'zz800_cyb_{benchmark_code}_gap.xlsx', engine='xlsxwriter') as writer:
    for i, weight_gap in enumerate(weight_gaps):
        ret_df = create_scenario(weight_gap, target_vol=0.01, method='risk_neutral')
        res_df = create_report(ret_df, 25)
        res_df.to_excel(writer, sheet_name=f'{i}')
        alpha_logger.info(f"weight_gap: {weight_gap} finished")

2019-02-10 00:34:13,667 - ALPHA_MIND - INFO - starting backting ...
2019-02-10 00:34:13,856 - ALPHA_MIND - INFO - alpha models training finished ...
D:\ProgramData\anaconda3\lib\site-packages\pandas\core\frame.py:4024: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
2019-02-10 00:34:13,921 - ALPHA_MIND - INFO - 2017-01-03 00:00:00 re-balance: 851 codes
2019-02-10 00:34:13,985 - ALPHA_MIND - INFO - 2017-01-17 00:00:00 re-balance: 850 codes
D:\ProgramData\anaconda3\lib\site-packages\cvxpy-1.0.10-py3.6-win-amd64.egg\cvxpy\problems\problem.py:614: RuntimeWarning: overflow encountered in long_scalars
  if self.max_big_small_squared < big*small**2:
D:\ProgramData\anaconda3\lib\site-packages\cvxpy-1.0.10-py3.6-win-amd64.egg\cvxpy\problems\problem.py:615: RuntimeWarning: overflow encountered in long_

2019-02-10 00:34:24,119 - ALPHA_MIND - INFO - 2017-08-01 00:00:00 re-balance: 853 codes
2019-02-10 00:34:24,250 - ALPHA_MIND - INFO - 2017-08-15 00:00:00 re-balance: 853 codes
2019-02-10 00:34:24,383 - ALPHA_MIND - INFO - 2017-08-29 00:00:00 re-balance: 853 codes
2019-02-10 00:34:24,513 - ALPHA_MIND - INFO - 2017-09-12 00:00:00 re-balance: 853 codes
2019-02-10 00:34:24,653 - ALPHA_MIND - INFO - 2017-09-26 00:00:00 re-balance: 853 codes
2019-02-10 00:34:24,774 - ALPHA_MIND - INFO - 2017-10-17 00:00:00 re-balance: 853 codes
2019-02-10 00:34:24,889 - ALPHA_MIND - INFO - 2017-10-31 00:00:00 re-balance: 854 codes
2019-02-10 00:34:25,036 - ALPHA_MIND - INFO - 2017-11-14 00:00:00 re-balance: 854 codes
2019-02-10 00:34:25,170 - ALPHA_MIND - INFO - 2017-11-28 00:00:00 re-balance: 854 codes
2019-02-10 00:34:25,298 - ALPHA_MIND - INFO - 2017-12-12 00:00:00 re-balance: 854 codes
2019-02-10 00:34:25,420 - ALPHA_MIND - INFO - 2017-12-26 00:00:00 re-balance: 854 codes
2019-02-10 00:34:25,531 - ALPHA_

Wall time: 17.4 s


In [5]:
%%time
target_vols = [0.015, 0.030, 0.045, 0.060]

with pd.ExcelWriter(f'zz800_cyb_{benchmark_code}_tv.xlsx', engine='xlsxwriter') as writer:
    for i, target_vol in enumerate(target_vols):
        ret_df = create_scenario(0.01, target_vol=target_vol, method='tv')
        res_df = create_report(ret_df, 25)
        res_df.to_excel(writer, sheet_name=f'{i}')
        alpha_logger.info(f"target_vol: {target_vol:.4f} finished")

2019-02-10 00:34:31,018 - ALPHA_MIND - INFO - starting backting ...
2019-02-10 00:34:31,080 - ALPHA_MIND - INFO - 2017-01-03 00:00:00 re-balance: 851 codes
2019-02-10 00:34:31,267 - ALPHA_MIND - INFO - 2017-01-17 00:00:00 re-balance: 850 codes
2019-02-10 00:34:31,465 - ALPHA_MIND - INFO - 2017-02-07 00:00:00 re-balance: 853 codes
2019-02-10 00:34:31,653 - ALPHA_MIND - INFO - 2017-02-21 00:00:00 re-balance: 852 codes
2019-02-10 00:34:31,866 - ALPHA_MIND - INFO - 2017-03-07 00:00:00 re-balance: 853 codes
2019-02-10 00:34:32,081 - ALPHA_MIND - INFO - 2017-03-21 00:00:00 re-balance: 853 codes
2019-02-10 00:34:32,284 - ALPHA_MIND - INFO - 2017-04-06 00:00:00 re-balance: 853 codes
2019-02-10 00:34:32,462 - ALPHA_MIND - INFO - 2017-04-20 00:00:00 re-balance: 853 codes
2019-02-10 00:34:32,660 - ALPHA_MIND - INFO - 2017-05-05 00:00:00 re-balance: 854 codes
2019-02-10 00:34:32,861 - ALPHA_MIND - INFO - 2017-05-19 00:00:00 re-balance: 854 codes
2019-02-10 00:34:33,047 - ALPHA_MIND - INFO - 2017-0

2019-02-10 00:34:45,521 - ALPHA_MIND - INFO - 2017-12-26 00:00:00 re-balance: 854 codes
2019-02-10 00:34:45,648 - ALPHA_MIND - INFO - 2018-01-10 00:00:00 re-balance: 856 codes
2019-02-10 00:34:45,800 - ALPHA_MIND - INFO - 2018-01-24 00:00:00 re-balance: 856 codes
2019-02-10 00:34:45,922 - ALPHA_MIND - INFO - 2018-02-07 00:00:00 re-balance: 856 codes
2019-02-10 00:34:46,036 - ALPHA_MIND - INFO - 2018-02-28 00:00:00 re-balance: 856 codes
2019-02-10 00:34:46,150 - ALPHA_MIND - INFO - 2018-03-14 00:00:00 re-balance: 856 codes
2019-02-10 00:34:46,257 - ALPHA_MIND - INFO - 2018-03-28 00:00:00 re-balance: 856 codes
2019-02-10 00:34:46,370 - ALPHA_MIND - INFO - 2018-04-13 00:00:00 re-balance: 856 codes
2019-02-10 00:34:46,483 - ALPHA_MIND - INFO - 2018-04-27 00:00:00 re-balance: 859 codes
2019-02-10 00:34:46,623 - ALPHA_MIND - INFO - target_vol: 0.0450 finished
2019-02-10 00:34:46,624 - ALPHA_MIND - INFO - starting backting ...
2019-02-10 00:34:46,685 - ALPHA_MIND - INFO - 2017-01-03 00:00:00 

Wall time: 19.3 s
